In [28]:
import requests
import json
import time
import pymongo
from bs4 import BeautifulSoup

In [10]:
# correcting university names
import pandas as pd
university_rankings = pd.read_json('university_rankings.json')

In [13]:
# correcting professor names
university_rankings.index

Index(['Boston University', 'California Institute of Technology',
       'Carnegie Mellon University', 'Colorado School of Mines',
       'Columbia University (Fu Foundation)', 'Cornell University',
       'Duke University (Pratt)', 'Emory University-Georgia Tech University',
       'Georgia Institute of Technology', 'Harvard University',
       'Iowa State University', 'Johns Hopkins University (Whiting)',
       'Louisiana State University--Baton Rouge',
       'Massachusetts Institute of Technology', 'Michigan State University',
       'North Carolina State University',
       'Northwestern University (McCormick)', 'Ohio State University',
       'Oregon State University',
       'Pennsylvania State University--University Park',
       'Princeton University', 'Purdue University--West Lafayette',
       'Rice University (Brown)', 'Stanford University',
       'Texas A&M University--College Station',
       'Texas Tech University (Whitacre)',
       'University of California--Berkeley

In [14]:
# Dictionary of school ids (keys) that map to corrected school name
sids = {124 : 'Boston University',
        181: 'Carnegie Mellon University', 
        'NA': 'Colorado School of Mines',
        278: 'Columbia University', 
        298: 'Cornell University',
        1350:'Duke University', 
        340: 'Emory University',
        361: 'Georgia Institute of Technology', 
        'NA': 'Harvard University',
        452: 'Iowa State University', 
        464: 'Johns Hopkins University',
        3071: 'Louisiana State University',
        580: 'Massachusetts Institute of Technology', 
        601: 'Michigan State University',
        685: 'North Carolina State University',
        709:'Northwestern University', 
        724: 'The Ohio State University',
        742:'Oregon State University',
        758:'Pennsylvania State University',
        780:'Princeton University', 
        783: 'Purdue University - West Lafayette',
        799: 'Rice University', 
        953: 'Stanford University',
       1003: 'Texas A&M University at College Station',
       1011: 'Texas Tech University',
       1072: 'University of California Berkeley', 
       1073:'University of California Davis',
       1079: 'University of California San Diego',
       1077: 'University of California Santa Barbara',
       1087: 'University of Colorado - Boulder', 
       1094: 'University of Delaware',
       1100: 'University of Florida', 
       1112:'University of Illinois at Urbana-Champaign',
       1117: 'University of Kansas', 
       1270: 'University of Maryland',
       1258: 'University of Michigan',
       1257: 'University of Minnesota - Twin Cities',
       1249: 'University of Nebraska - Lincoln', 
       1596: 'University of Oklahoma',
       1275:'University of Pennsylvania',
       1381: 'University of Southern California',
       1385: 'University of Tennessee - Knoxville',
       1255: 'University of Texas at Austin', 
       3963:'University of Tulsa',
       1530: 'University of Washington', 
       1256: 'University of Wisconsin - Madison',
       4282: 'University of Wyoming', 
       1349: 'Virginia Tech', 
       1222: 'Yale University'}

In [149]:
with open('sids.json', 'r') as f:
    sids = json.load(f)

with open('utpge.json', 'r') as f:
    ut = json.load(f)

with open('university_rankings.json', 'r') as f:
    university_rankings = json.load(f)

In [144]:
# final code
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

def rating_and_tags(sid, school_name, faculty_name):
    '''Gives rating and tags for a professor pulled from ratemyprofessors.com
       given sid (school's id as per ratemyprofessors.com), school_name and faculty_name
    '''
        
     # Getting rating url by searching school and faculty name
    url = f"http://www.ratemyprofessors.com/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName={school_name}&schoolID={sid}&query={faculty_name}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    try:
        rating_url = "http://www.ratemyprofessors.com" + soup.select("li.listing.PROFESSOR")[0].find("a")["href"]

        # Getting rating and profile tags for the rating url
        response2 = requests.get(rating_url, headers=headers)
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        rating = float(soup2.select("div.grade")[0].get_text())
        tags = [tag_desc.get_text() for tag_desc in soup2.select("span.tag-box-choosetags")]
        
    except:
        rating, tags = 'N/A', 'N/A'

    return {'rating': rating, 'tags': tags}

In [151]:
university_rankings['petroleum-engineering']

{'University of Texas--Austin (Cockrell)': {'location': 'Austin, TX',
  'score': 4.7,
  'rank': 1},
 'Stanford University': {'location': 'Stanford, CA', 'score': 4.3, 'rank': 2},
 'Texas A&M University--College Station': {'location': 'College Station, TX',
  'score': 4.3,
  'rank': 2},
 'University of Tulsa': {'location': 'Tulsa, OK', 'score': 3.9, 'rank': 4},
 'Colorado School of Mines': {'location': 'Golden, CO',
  'score': 3.8,
  'rank': 5},
 'Pennsylvania State University--University Park': {'location': 'University Park, PA',
  'score': 3.5,
  'rank': 6},
 'University of Oklahoma': {'location': 'Norman, OK', 'score': 3.4, 'rank': 7},
 'University of Southern California (Viterbi)': {'location': 'Los Angeles, CA',
  'score': 3.1,
  'rank': 8},
 'Louisiana State University--Baton Rouge': {'location': 'Baton Rouge, LA',
  'score': 2.5,
  'rank': 9},
 'Texas Tech University (Whitacre)': {'location': 'Lubbock, TX',
  'score': 2.5,
  'rank': 9},
 'University of Kansas': {'location': 'Lawr

In [154]:
for faculty in ut['faculty_names']:
    university_rankings['petroleum-engineering']['University of Texas--Austin (Cockrell)'] [faculty] = rating_and_tags(sid, school, faculty)

university_rankings['petroleum-engineering']

{'University of Texas--Austin (Cockrell)': {'location': 'Austin, TX',
  'score': 4.7,
  'rank': 1,
  'rating': 'N/A',
  'tags': 'N/A',
  'Matthew Balhoff': {'rating': 'N/A', 'tags': 'N/A'},
  'J.Eric Bickel': {'rating': 'N/A', 'tags': 'N/A'},
  'Paul Bommer': {'rating': 'N/A', 'tags': 'N/A'},
  'Hugh Daigle': {'rating': 'N/A', 'tags': 'N/A'},
  'Mojdeh Delshad': {'rating': 'N/A', 'tags': 'N/A'},
  'David DiCarlo': {'rating': 3.5,
   'tags': [' Tough Grader (2)',
    ' TEST HEAVY (1)',
    ' LOTS OF HOMEWORK (1)',
    ' Hilarious (1)',
    ' Inspirational (1)']},
  'Nicolas Espinoza': {'rating': 'N/A', 'tags': 'N/A'},
  'John Foster': {'rating': 'N/A', 'tags': 'N/A'},
  'Kenneth Gray': {'rating': 'N/A', 'tags': 'N/A'},
  'Zoya Heidari': {'rating': 'N/A', 'tags': 'N/A'},
  'Deborah Hempel-Medina': {'rating': 4.0,
   'tags': [' Respected (1)',
    ' Participation matters (1)',
    ' Gives good feedback (1)']},
  'Chun Huh': {'rating': 'N/A', 'tags': 'N/A'},
  'Larry Lake': {'rating': 2.4,